In [3]:
%load_ext autoreload
%autoreload 2


from glob import glob
from itertools import chain, combinations
import math
import re

# import ensembl_rest
import numpy as np
import pandas as pd
from sourmash import load_signatures

import requests, sys
from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
 
from sklearn.metrics import auc, roc_curve

import tqdm

from sklearn import metrics

import seaborn as sns
import vaex

from khtools.ensembl import get_sequence, maybe_get_cds


from khtools.compare_peptide import compare_all_seqs
from khtools.homology import HomologyTable



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%%time

output_folder = '/home/olga/data_sm/kmer-hashing/orthologues/human-vs-mouse'
n_subset = 10000
protein_coding_peptide_orthology = pd.read_csv(f"{output_folder}/protein_coding_peptide_orthology_nsubset{n_subset}_seed0.csv", index_col=0)
print(protein_coding_peptide_orthology.shape)
protein_coding_peptide_orthology.head()

/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(198625830, 29)
CPU times: user 15min 26s, sys: 7min 57s, total: 23min 23s
Wall time: 23min 26s


In [8]:
protein_coding_peptide_orthology.head()

,id1,id2,ksize,jaccard,molecule,species1,species2,species_species,Gene stable ID,Gene stable ID version,...,Last common ancestor with Mouse,Mouse homology type,%id. target Mouse gene identical to query gene,%id. query gene identical to target Mouse gene,Mouse Gene-order conservation score,Mouse Whole-genome alignment coverage,dN with Mouse,dS with Mouse,"Mouse orthology confidence [0 low, 1 high]",is_homologue
0,ENSP00000482393,ENSMUSP00000022095,2,0.909091,protein,human,mouse,human_mouse,ENSG00000113430,ENSG00000113430.10,...,Eutheria,ortholog_one2one,77.7982,82.3301,100.0,100.0,0.0881,1.0015,1.0,True
1,ENSP00000482393,ENSMUSP00000022095,3,0.678879,protein,human,mouse,human_mouse,ENSG00000113430,ENSG00000113430.10,...,Eutheria,ortholog_one2one,77.7982,82.3301,100.0,100.0,0.0881,1.0015,1.0,True
2,ENSP00000482393,ENSMUSP00000022095,4,0.582178,protein,human,mouse,human_mouse,ENSG00000113430,ENSG00000113430.10,...,Eutheria,ortholog_one2one,77.7982,82.3301,100.0,100.0,0.0881,1.0015,1.0,True
3,ENSP00000482393,ENSMUSP00000022095,5,0.523529,protein,human,mouse,human_mouse,ENSG00000113430,ENSG00000113430.10,...,Eutheria,ortholog_one2one,77.7982,82.3301,100.0,100.0,0.0881,1.0015,1.0,True
4,ENSP00000482393,ENSMUSP00000022095,6,0.482353,protein,human,mouse,human_mouse,ENSG00000113430,ENSG00000113430.10,...,Eutheria,ortholog_one2one,77.7982,82.3301,100.0,100.0,0.0881,1.0015,1.0,True
